# <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea"> Capstone Project


### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#da614e"> Capstone Overview


## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> Objective:

- To undertake a multi-faceted project that demonstrates your understanding and mastery of the key conceptual and technological aspects of Deep Learning.
- To develop an understanding of how challenging human-level problems can be approached and solved using a combination of tools and techniques.
- To understand current scenarios in deep learning, understand the practicalities and the trade-offs that need to be made when solving a problem in real life.
  ###Milestones:

## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> This project has 2 milestones.

You get 3 weeks for completing each milestone.
The 1st milestone should be submitted as part of the interim report.

### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#da614e"> 1st Milestone:

## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> 1. Problem interpretation

- Understand the data
- Make an abstract or an overview based on your approach
- Break the problem into smaller tasks
- Discuss among your teammates and share responsibilities

## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> 2. Data analysis and preprocessing:

Visual displays are powerful when used well, so think carefully about the information the display.

- Include any insightful visualization
- Share and explain particularly meaningful features, interactions or summary of data
- Display examples to input in your model
- Explain changes to be incorporated into data so that it becomes ready for the model

## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> 3. Modeling

- What kind of neural network you have used and why?
- What progress you have made towards your intended solution?


### <span style="font-family: Arial; font-weight:bold;font-size:1.2em;color:#da614e"> 2nd Milestone

## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> 4. Model evaluation

- Describe how you will proceed with the analysis
- Compare different models and choose which model to use
- Do hyper-parameter tuning of your model
- How will you build on your initial analysis to increase the accuracy of your model?

## <span style="font-family: Arial; font-weight:normal;font-size:0.5em;color:#0e92ea"> 5. Presentation and Report

- You should start preparing the final report at least 2 weeks prior to the project completion date.
- Teams should send a draft Last of the project before the last session to the mentor and get the necessary inputs for submission.
- The expectations for the final report will be included in your Capstone course page


In [ ]:
# Importing necessary libraries

import os
import numpy as np
import pandas as pd
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns
import math
from skimage import measure
from skimage.transform import resize